<a href="https://colab.research.google.com/github/ABD-01/MNIST/blob/main/MNIST_Using_Multi_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>